Source:
https://learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

Resources:
* https://github.com/pydata/pandas-datareader
* https://h1ros.github.io/posts/getting-real-time-stock-market-data/

for future project
* https://hackernoon.com/scraping-yahoo-finance-data-using-python-ayu3zyl
* https://medium.com/harinathselvaraj/python-code-to-get-realtime-stock-prices-from-yahoo-finance-66d7d1858133

In [1]:
!pip install pandas-datareader

     |████████████████████████████████| 107 kB 460 kB/s eta 0:00:01


In [2]:
# importing pandas-datareader webscrapper
import pandas_datareader as pdr

In [3]:
# Defining wich instruments I want to track. For now I would try only with the mexican index IPC 
tickers = ['MXX']

In [10]:
# Let's try with last week data
start_date = '2020-07-10'
end_date = '2020-07-17'

In [13]:
# Let's define the source
source = 'yahoo'

In [19]:
# Let's fetch the data
panel_data = pdr.DataReader('^MXX','yahoo', start_date, end_date)

In [21]:
# Let's see the fetched data
panel_data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-07-10,36777.769531,36263.968750,36777.769531,36465.460938,105150800,36465.460938
2020-07-13,36939.691406,36321.589844,36602.769531,36389.390625,150410600,36389.390625
2020-07-14,36365.769531,35883.468750,36365.769531,36190.910156,166784200,36190.910156
2020-07-15,36717.531250,36427.839844,36427.839844,36590.261719,194173400,36590.261719
2020-07-16,36533.558594,36193.718750,36517.261719,36465.671875,163886500,36465.671875
2020-07-17,36589.058594,36172.988281,36553.710938,36327.839844,120832098,36327.839844


ok, this is good, but only works for historical data

In [22]:
!pip install yahoo_fin

In [24]:
!pip install requests_html

     |████████████████████████████████| 145 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 1.8 MB/s  eta 0:00:01
  Created wheel for parse: filename=parse-1.15.0-py3-none-any.whl size=23709 sha256=cb2b0582e39964e0db595d673a0d8a19710a79123e2106ee503d11aef202e097
  Stored in directory: /home/santos/.cache/pip/wheels/d7/b3/1d/5c94c64413b2212f64a297c92f11edd45e4474d08d0220a008
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=02757cbbad54cfc1816a3657366a6141f30beb851e461e20db88e4f45f20f2b8
  Stored in directory: /home/santos/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13487 sha256=3f90020731b60b5aaf281ea98e13160437ce4452933a884db87153c49c179af9
  Stored in directory: /home/santos/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built parse bs4 fake-useragent


In [32]:
# importing yahoo_fin lib
from yahoo_fin import stock_info

In [64]:
stock_info.get_live_price('^mxx')

36327.83984375

In [65]:
value =stock_info.get_live_price('^mxx')

seems to work fine

now, let's connect to influx db and save this value at '^mxx-day'

In [2]:
!pip install influxdb

     |████████████████████████████████| 77 kB 436 kB/s eta 0:00:011
     |████████████████████████████████| 118 kB 1.9 MB/s eta 0:00:01
  Created wheel for msgpack: filename=msgpack-0.6.1-cp38-cp38-linux_x86_64.whl size=343714 sha256=ab74c109c97bb8d20c25a0af652b897853635a6173d0299ddd152a43e1e4d07b
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/2e/45/9ae160fc31c10e4b799d0ebc32ba82b32f1b057e0ebf28ea82
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.0
    Uninstalling msgpack-1.0.0:
      Successfully uninstalled msgpack-1.0.0


In [8]:
from influxdb import InfluxDBClient
from influxdb import exceptions

In [10]:
try:
    client = InfluxDBClient('172.18.0.2', 8086)
    client.ping()
    print('Connected to InfluxDB')
except Exception as e:
    print('Connection error: {}'.format(str(e)))

Connected to InfluxDB


In [43]:
client.get_list_database()

[{'name': '_internal'}, {'name': 'testdb'}]

In [45]:
client.switch_database('testdb')

In [57]:
import datetime

In [70]:
# from influx db python documentation
json_body = [
    {
        "measurement": "mxx",
        "tags": {
            "measurement_type": "day",
            "source": "yahoo"
        },
        "time": '{}'.format(datetime.datetime.now()),
        "fields": {
            "value": value
        }
    }
]


In [71]:
json_body

[{'measurement': 'mxx',
  'tags': {'measurement_type': 'day', 'source': 'yahoo'},
  'time': '2020-07-17 18:00:13.389995',
  'fields': {'value': 36327.83984375}}]

In [72]:
# write_points function also can take de different fields as parameters. See documentation.

client.write_points(json_body, database='testdb')

True

In [73]:
client.get_list_measurements()

[{'name': 'cpu_load_short'}, {'name': 'mxx'}]

In [81]:
client.query("Select \"value\" from mxx where \"measurement_type\" = 'day'")

ResultSet({'('mxx', None)': [{'time': '2020-07-17T18:00:13.389995Z', 'value': 36327.83984375}]})

worked

## Let's wrap all this process together

In [96]:
import pytz

In [100]:
# importing yahoo_fin lib
from yahoo_fin import stock_info
from influxdb import InfluxDBClient
from influxdb import exceptions
from datetime import datetime

# from influx db python documentation
json_body = [
    {
        "measurement": "mxx",
        "tags": {
            "measurement_type": "day",
            "source": "yahoo"
        },
        "time": None,
        "fields": {
            "value": None
        }
    }
]

tz = pytz.timezone('America/Mexico_City')

try:
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    value = stock_info.get_live_price('^mxx')
    json_body[0]['time'] = '{}'.format(datetime.now(tz))
    json_body[0]['fields']['value'] = value
    client.write_points(json_body, database='testdb')
except Exception as e:
    print('Connection error: {}'.format(str(e)))
    

In [101]:
client.query("Select \"value\" from mxx where \"measurement_type\" = 'day'", database = 'testdb')

ResultSet({'('mxx', None)': [{'time': '2020-07-17T18:00:13.389995Z', 'value': 36327.83984375}, {'time': '2020-07-17T18:35:43.009230Z', 'value': 36327.83984375}, {'time': '2020-07-17T23:40:32.066297Z', 'value': 36327.83984375}]})

Again, it works

### let's create a never ending process here with logging

In [2]:
%%writefile yahoo_service.py
# importing yahoo_fin lib
from yahoo_fin import stock_info
from influxdb import InfluxDBClient
from influxdb import exceptions
from datetime import datetime
import logging
import time
import pytz

logging.basicConfig(filename='service.log',level=logging.INFO, format='%(asctime)s %(message)s')

# from influx db python documentation
json_body = [
    {
        "measurement": "mxx",
        "tags": {
            "measurement_type": "day",
            "source": "yahoo"
        },
        "time": None,
        "fields": {
            "value": None
        }
    }
]

tz = pytz.timezone('America/Mexico_City')
logging.info('Setting timezone to America/Mexico_City')

count=0
count_threshold=10
try:
    logging.info('Connecting to InfluxDB using port 8086...')
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    logging.info('Connected!')
    logging.info('Reading stocks value!')
    while(True):
        value = stock_info.get_live_price('^mxx')
        json_body[0]['time'] = '{}'.format(datetime.now(tz))
        json_body[0]['fields']['value'] = value
        client.write_points(json_body, database='testdb')
        time.sleep(1)
        count=count+1
        if(count==count_threshold):
            count = 0 
            logging.info('Writing without problems')
except Exception as e:
    print('Error: {}'.format(str(e)))
    logging.error('Service stoped')
    logging.error(e)
    

Overwriting yahoo_service.py


In [1]:
%run -i yahoo_service.py

KeyboardInterrupt: 

### Now I have a raw service to read and write stock data from yahoo API to InfluxDB but it will work 24/7 and I don't want that

In [1]:
# 1. check day is weekday
import datetime

In [8]:
weekday = datetime.datetime.now().weekday()
if(weekday in [5,6]):
    print('Is weekend')

Is weekend


In [11]:
# 2. Check if is in laboral hours:
date =  datetime.datetime.now()

In [17]:
if (9<=date.hour<18):
    print('working time')

working time


### let's add it

In [27]:
%%writefile yahoo_service.py
# importing yahoo_fin lib
from yahoo_fin import stock_info
from influxdb import InfluxDBClient
from influxdb import exceptions
from datetime import datetime
import logging
import time
import pytz

logging.basicConfig(filename='service.log',level=logging.INFO, format='%(asctime)s %(message)s')

# from influx db python documentation

tz = pytz.timezone('America/Mexico_City')
logging.info('Setting timezone to America/Mexico_City')
count=0
count_threshold=10
try:
    logging.info('Connecting to InfluxDB using port 8086...')
    client = InfluxDBClient('localhost', 8086)
    client.ping()
    logging.info('Connected!')
    logging.info('Reading stocks value...')
    while(True):
        if (9<=date.hour<18):
            value = stock_info.get_live_price('^mxx')
            json_body[0]['time'] = '{}'.format(datetime.now(tz))
            json_body[0]['fields']['value'] = value
            client.write_points(json_body, database='testdb')
            time.sleep(1)
            count=count+1
            if(count==count_threshold):
                count = 0 
                logging.info('Writing without problems')
        else:
            logging.info('BMV finished operations for today.')
            logging.info('Closing connections.')
            break
except Exception as e:
    print('Error: {}'.format(str(e)))
    logging.error('Service stoped')
    logging.error(e)
    

Overwriting yahoo_service.py


### Adding configuration file

In [22]:
import pytz
tz = pytz.timezone('America/Mexico_City')

In [11]:
%%writefile config.py
import logging
import pytz

log_config={
    'log_file_name' : 'yahoo_service.log',
    'logging_level' : logging.INFO,
    'logging_format' : '%(asctime)s %(message)s' ,
}

influxdb_config={
    
    'influxdb_host' : '172.18.0.2', #check the ipv4 from docker network
    'influxdb_port' : 8086,
    'influxdb_db' : 'testdb',
    'influxdb_json_body' : {
        "measurement": "mxx",
        "tags": {
            "measurement_type": "day",
            "source": "yahoo"
        },
        "time": None,
        "fields": {
            "value": None
        }
    },
}

script_config={    
    'timezone' : pytz.timezone('America/Mexico_City'),
    'script_count_threshold':10
}

Writing config.py


### Let's add the config file into the original code

In [12]:
%%writefile yahoo_service.py
# importing yahoo_fin lib
from influxdb import InfluxDBClient, exceptions
from yahoo_fin import stock_info
from datetime import datetime
import logging
import config
import time
import pytz

logging.basicConfig(filename=config.log_config.get('log_file_name'),
                    level=config.log_config.get('logging_level'),
                    format=config.log_config.get('logging_format')
                   )

# from influx db python documentation


tz = config.script_config.get('timezone')
logging.info('Setting timezone to {}'.format(tz.zone))
count=0
count_threshold=config.script_config.get('script_count_threshold')
try:
    logging.info('Connecting to InfluxDB using port {}...'.format(config.influxdb_config.get('influxdb_port')))
    client = InfluxDBClient(config.influxdb_config.get('influxdb_host'),
                            config.influxdb_config.get('influxdb_port')
                           )
    client.ping()
    client.switch_database(config.influxdb_config.get('influxdb_db'))
    json_body = [config.influxdb_config.get('influxdb_json_body')]
    logging.info('Connected!')
    logging.info('Reading stocks value...')
    while(True):
        if (9<=datetime.now().hour<18):
            value = stock_info.get_live_price('^mxx')
            json_body[0]['time'] = '{}'.format(datetime.now(tz))
            json_body[0]['fields']['value'] = value
            client.write_points(json_body)
            time.sleep(1)
            count=count+1
            if(count==count_threshold):
                count = 0 
                logging.info('Writing without problems')
        else:
            logging.info('BMV finished operations for today.')
            logging.info('Closing connections.')
            break
except Exception as e:
    print('Error: {}'.format(str(e)))
    logging.error('Service stoped')
    logging.error(e)
    

Overwriting yahoo_service.py


In [1]:
from influxdb import InfluxDBClient
from yahoo_fin import stock_info
from influxdb import exceptions
from datetime import datetime
import logging
import config
import time
import pytz

logging.basicConfig(filename=config.log_config.get('log_file_name'),
                    level=config.log_config.get('logging_level'),
                    format=config.log_config.get('logging_format')
                   )

In [3]:
tz = config.script_config.get('timezone')
logging.info('Setting timezone to {}'.format(tz.zone))
count=0
count_threshold=config.script_config.get('script_count_threshold')

In [11]:
    logging.info('Connecting to InfluxDB using port {}...'.format(config.influxdb_config.get('influxdb_port')))
    client = InfluxDBClient(config.influxdb_config.get('influxdb_host'),
                            config.influxdb_config.get('influxdb_port')
                           )
    client.ping()
    client.switch_database(config.influxdb_config.get('influxdb_db'))
    json_body = [config.influxdb_config.get('influxdb_json_body')]
    logging.info('Connected!')
    logging.info('Reading stocks value...')

In [12]:
json_body

[{'measurement': 'mxx',
  'tags': {'measurement_type': 'day', 'source': 'yahoo'},
  'time': '2020-07-18 17:35:29.854227-05:00',
  'fields': {'value': 36327.83984375}}]

In [31]:
if (9<=datetime.now().hour<18):
            value = stock_info.get_live_price('^mxx')
            json_body[0]['time'] = '{}'.format(datetime.now(tz))
            json_body[0]['fields']['value'] = value
            client.write_points(json_body)
            time.sleep(1)
            count=count+1
            if(count==count_threshold):
                count = 0 
                logging.info('Writing without problems')
else:
            logging.info('BMV finished operations for today.')
            logging.info('Closing connections.')

### seems to work without problems